# Tiền xử lí dữ liệu

In [87]:
# PRE_PROCESS_SRC_PATH = "./srcPreprocessingtomcat.csv"
# PRE_PROCESS_REPORT_PATH = "./ReportPreprocessingtomcat.csv"
# REPORT_PATH = "./Tomcat.xlsx"

# PRE_PROCESS_SRC_PATH = "./srcPreprocessingswt.csv"
# PRE_PROCESS_REPORT_PATH = "./ReportPreprocessingswt.csv"
# REPORT_PATH = "./SWT.xlsx"

PRE_PROCESS_SRC_PATH = "./srcPreprocessingaspectj.csv"
PRE_PROCESS_REPORT_PATH = "./ReportPreprocessingaspectj.csv"
REPORT_PATH = "./AspectJ.xlsx"

In [88]:
import pandas as pd

pre_process_report_path = PRE_PROCESS_REPORT_PATH
pre_process_report_df = pd.read_csv(pre_process_report_path)
pre_process_report_df.head()

,key,summary,description,fixed_files,report_time,pos_tagged_summary,pos_tagged_description,stack_traces,stack_traces_remove
0,423257,"{'stemmed': ['bug', 'ltw', 'java', 'lang', 've...","{'stemmed': [], 'unstemmed': []}",['org.aspectj.ajdt.core\\src\\org\\aspectj\\aj...,2013-12-04 19:43:22,"{'stemmed': ['bug', 'ltw', 'bad', 'type', 'gen...","{'stemmed': [], 'unstemmed': []}",[],[]
1,420210,"{'stemmed': ['bug', 'support', 'addit', 'messa...","{'stemmed': ['would', 'good', 'abl', 'insert',...",['org.aspectj.matcher\\src\\org\\aspectj\\weav...,2013-10-23 16:00:16,"{'stemmed': ['support', 'messag', 'key', 'decl...","{'stemmed': ['insert', 'enclos', 'name', 'encl...",[],[]
2,419279,"{'stemmed': ['bug', 'ajc', 'option', 'chang', ...","{'stemmed': ['xlintfil', 'option', 'great', 'f...",['org.aspectj.ajdt.core\\src\\org\\aspectj\\aj...,2013-10-11 15:20:08,"{'stemmed': ['bug', 'ajc', 'option', 'chang', ...","{'stemmed': ['option', 'fit', 'control', 'mess...",[],[]
3,415266,"{'stemmed': ['bug', 'ltw', 'work', 'jmx', 'ena...","{'stemmed': ['enabl', 'jmx', 'remot', 'manag',...",['loadtime\\src\\org\\aspectj\\weaver\\loadtim...,2013-08-16 17:51:29,"{'stemmed': ['bug', 'ltw', 'work', 'jmx', 'ena...","{'stemmed': ['enabl', 'manag', 'jvm', 'aspectj...",[],[]
4,418129,"{'stemmed': ['bug', 'introduc', 'annot', 'onto...","{'stemmed': [], 'unstemmed': []}","['tests\\bugs174\\pr418129\\Target.java', ' te...",2013-09-26 14:31:09,"{'stemmed': ['bug', 'ca', 'introduc', 'annot',...","{'stemmed': [], 'unstemmed': []}",[],[]


In [89]:
import ast

list_groud_truth = []
list_query = []
report_dict = {}

for index, row in pre_process_report_df.iterrows():
    groud_truth = ast.literal_eval(row['fixed_files'])
    query_summary = ast.literal_eval(row['summary'])
    query_summary = query_summary['stemmed']

    query_des = ast.literal_eval(row['description'])
    query_des = query_des['stemmed']

    query = query_summary + query_des

    list_query.append(query)
    list_groud_truth.append(groud_truth)

# Tạo dictionary từ lists và keys
for index, row in pre_process_report_df.iterrows():
    key = row['key']  # Sử dụng trực tiếp giá trị của row['key']
    report_dict[key] = {
        'groud_truth': list_groud_truth[index],
        'query': list_query[index]
    }

print(len(list_groud_truth))
print(len(list_query))
print(report_dict)

593
593
{423257: {'groud_truth': ['org.aspectj.ajdt.core\\src\\org\\aspectj\\ajdt\\internal\\core\\builder\\AjState.java', ' tests\\bugs175\\pr423257\\AspectX.java', ' tests\\bugs175\\pr423257\\Test.java', ' tests\\src\\org\\aspectj\\systemtest\\AllTests17.java', ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\Ajc175Tests.java', ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\AllTestsAspectJ175.java', ' tests\\src\\org\\aspectj\\systemtest\\incremental\\tools\\MultiProjectIncrementalTests.java', ' weaver\\src\\org\\aspectj\\weaver\\bcel\\asm\\StackMapAdder.java', '.'], 'query': ['bug', 'ltw', 'java', 'lang', 'verifyerror', 'bad', 'type', 'gener', 'local', 'variabl', 'verifi', 'error']}, 420210: {'groud_truth': ['org.aspectj.matcher\\src\\org\\aspectj\\weaver\\Checker.java', ' tests\\bugs174\\extra_inserts\\Code.java', ' tests\\src\\org\\aspectj\\systemtest\\ajc174\\Ajc174Tests.java', '.'], 'query': ['bug', 'support', 'addit', 'messag', 'insert', 'key', 'declar', 'error', 'warn', 'would

# Tạo chỉ mục ngược cho all contents trong source và tìm kiếm bằng VSM

In [90]:
import pandas as pd
import math
from collections import Counter

# Đọc file CSV
file_path = PRE_PROCESS_SRC_PATH
df = pd.read_csv(file_path)

# Hàm chuẩn bị dữ liệu
def prepare_data(df):
    data = []
    for index, row in df.iterrows():
        all_content = eval(row['all_content'])
        data.append({
            "key": row['key'],
            "all_content": {
                "stemmed": all_content['stemmed']
            }
        })
    return data

# Hàm xây dựng chỉ mục ngược
def build_inverted_index(data):
    inverted_index = {}
    for item in data:
        file_path = item["key"]
        stemmed_words = item["all_content"]["stemmed"]
        for word in stemmed_words:
            if word not in inverted_index:
                inverted_index[word] = []
            if file_path not in inverted_index[word]:
                inverted_index[word].append(file_path)
    return inverted_index

# Hàm xây dựng từ vựng
def build_vocabulary(inverted_index):
    vocabulary = {word: idx for idx, word in enumerate(inverted_index.keys())}
    return vocabulary

# Hàm xây dựng VSM thưa với TF-IDF dạng ltc (cho tài liệu)
def build_sparse_vsm_ltc(data, inverted_index, vocabulary):
    N = len(data)
    sparse_vsm = {}
    for item in data:
        doc_id = item["key"]
        words = item["all_content"]["stemmed"]
        term_freq = Counter(words)
        tfidf_vector = {}
        for word, freq in term_freq.items():
            tf = 1 + math.log(freq)
            df_t = len(inverted_index.get(word, []))
            idf = math.log(N / df_t) if df_t > 0 else 0
            tfidf_vector[word] = tf * idf
        norm = math.sqrt(sum(w ** 2 for w in tfidf_vector.values()))
        if norm > 0:
            for word in tfidf_vector:
                tfidf_vector[word] /= norm
        sparse_vector = {vocabulary[word]: weight for word, weight in tfidf_vector.items()}
        sparse_vsm[doc_id] = sparse_vector
    return sparse_vsm

# Hàm tính TF-IDF lnc cho query
def compute_query_lnc(query, vocabulary):
    term_freq = Counter(query)
    tf_vector = {}
    for word, freq in term_freq.items():
        if word in vocabulary:  # Chỉ tính với từ có trong từ vựng
            tf_vector[word] = 1 + math.log(freq)  # l: logarithmic TF
            # Không nhân IDF (n: no IDF)
    norm = math.sqrt(sum(w ** 2 for w in tf_vector.values()))
    if norm > 0:
        for word in tf_vector:
            tf_vector[word] /= norm  # c: cosine normalization
    sparse_query = {vocabulary[word]: weight for word, weight in tf_vector.items()}
    return sparse_query

# Hàm tính cosine similarity giữa hai vector thưa
def cosine_similarity(vec1, vec2):
    dot_product = 0
    for idx in vec1:
        if idx in vec2:
            dot_product += vec1[idx] * vec2[idx]
    return dot_product

# Hàm tìm top k tài liệu tương đồng nhất
def find_top_k_similar(query, sparse_vsm, vocabulary, k=5):
    query_vector = compute_query_lnc(query, vocabulary)
    similarities = {}

    for doc_id, doc_vector in sparse_vsm.items():
        sim = cosine_similarity(query_vector, doc_vector)
        similarities[doc_id] = sim

    top_k = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:k]

    return top_k

# Chuẩn bị dữ liệu và xây dựng VSM
dataSrc = prepare_data(df)
inverted_index = build_inverted_index(dataSrc)
vocabulary = build_vocabulary(inverted_index)
sparse_vsm_ltc = build_sparse_vsm_ltc(dataSrc, inverted_index, vocabulary)

# Ví dụ: Query là một danh sách từ
# query = ['bug', 'wrong', 'statement', 'gener', 'use', 'attribut', 'page', 'direct']  # Thay bằng danh sách từ của bạn


In [91]:
list_prediction = {}
for key, value in report_dict.items():
    query = value['query']
    top_k = find_top_k_similar(query, sparse_vsm_ltc, vocabulary, k=100)
    list_prediction[key] = top_k

# Lấy mảng các value từ list_prediction
prediction_values = list(list_prediction.values())
# In kết quả
# print(f"Query: {query}")
# print("Top 3 similar documents:")
# i = 0
# for doc_id, sim in top_k:
#   i += 1
#   # print(type(doc_id))
#   if doc_id == 'java\\org\\apache\\jasper\\compiler\\PageInfo.java':
#     print(i)
#     print(f"Document: {doc_id}, Similarity: {sim:.4f}")

In [92]:
prediction_values = list(list_prediction.values())

In [93]:
# list_prediction[56012]

# Evaluation khi chưa chọn danh sách file có liên quan

In [94]:
# ground_truth = [['java\\org\\apache\\jasper\\compiler\\PageInfo.java', ' java\\org\\apache\\jasper\\compiler\\Validator.java', '.']]


# predictions = [
#     list_prediction[0]
# ]

ground_truth = list_groud_truth
predictions = prediction_values

In [95]:
predictions[0]

[]

In [96]:
ground_truth[0]

['org.aspectj.ajdt.core\\src\\org\\aspectj\\ajdt\\internal\\core\\builder\\AjState.java',
 ' tests\\bugs175\\pr423257\\AspectX.java',
 ' tests\\bugs175\\pr423257\\Test.java',
 ' tests\\src\\org\\aspectj\\systemtest\\AllTests17.java',
 ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\Ajc175Tests.java',
 ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\AllTestsAspectJ175.java',
 ' tests\\src\\org\\aspectj\\systemtest\\incremental\\tools\\MultiProjectIncrementalTests.java',
 ' weaver\\src\\org\\aspectj\\weaver\\bcel\\asm\\StackMapAdder.java',
 '.']

In [97]:
len(ground_truth)

593

In [98]:
len(predictions)

593

In [99]:
ground_truth[0]


['org.aspectj.ajdt.core\\src\\org\\aspectj\\ajdt\\internal\\core\\builder\\AjState.java',
 ' tests\\bugs175\\pr423257\\AspectX.java',
 ' tests\\bugs175\\pr423257\\Test.java',
 ' tests\\src\\org\\aspectj\\systemtest\\AllTests17.java',
 ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\Ajc175Tests.java',
 ' tests\\src\\org\\aspectj\\systemtest\\ajc175\\AllTestsAspectJ175.java',
 ' tests\\src\\org\\aspectj\\systemtest\\incremental\\tools\\MultiProjectIncrementalTests.java',
 ' weaver\\src\\org\\aspectj\\weaver\\bcel\\asm\\StackMapAdder.java',
 '.']

In [100]:
import numpy as np

# Hàm tính Top-k Accuracy
def top_k_accuracy(predictions, ground_truth, k=1):
    correct = 0
    total_queries = len(predictions)

    for query_idx in range(total_queries):
        pred_docs = [doc_id for doc_id, _ in predictions[query_idx][:k]]  # Lấy top k tài liệu dự đoán
        true_docs = ground_truth[query_idx]

        # Kiểm tra xem có ít nhất một tài liệu đúng trong top k không
        if any(doc in true_docs for doc in pred_docs):
            correct += 1

    return correct / total_queries if total_queries > 0 else 0

# Hàm tính Average Precision cho một truy vấn
def average_precision(predicted, relevant):
    if not relevant:
        return 0.0

    relevant_set = set(relevant)
    hits = 0
    precision_sum = 0.0

    for i, (doc_id, _) in enumerate(predicted):
        if doc_id in relevant_set:
            hits += 1
            precision_sum += hits / (i + 1)  # Precision tại vị trí i

    return precision_sum / len(relevant) if hits > 0 else 0.0

# Hàm tính MAP (Mean Average Precision)
def mean_average_precision(predictions, ground_truth):
    ap_scores = []
    total_queries = len(predictions)

    for query_idx in range(total_queries):
        pred_docs = predictions[query_idx]  # Danh sách (doc_id, similarity)
        true_docs = ground_truth[query_idx]
        ap = average_precision(pred_docs, true_docs)
        ap_scores.append(ap)

    return np.mean(ap_scores) if ap_scores else 0.0

# Hàm tính MRR (Mean Reciprocal Rank)
def mean_reciprocal_rank(predictions, ground_truth):
    rr_scores = []
    total_queries = len(predictions)

    for query_idx in range(total_queries):
        pred_docs = [doc_id for doc_id, _ in predictions[query_idx]]  # Danh sách tài liệu dự đoán
        true_docs = set(ground_truth[query_idx])

        for rank, doc_id in enumerate(pred_docs, 1):
            if doc_id in true_docs:
                rr_scores.append(1.0 / rank)
                break

    return np.mean(rr_scores) if rr_scores else 0.0

# Ví dụ sử dụng
# Giả sử bạn có ground truth và predictions như sau



In [101]:
# Tính các chỉ số
top1_acc = top_k_accuracy(predictions, ground_truth, k=1)
top5_acc = top_k_accuracy(predictions, ground_truth, k=5)
top10_acc = top_k_accuracy(predictions, ground_truth, k=10)
top20_acc = top_k_accuracy(predictions, ground_truth, k=20)
map_score = mean_average_precision(predictions, ground_truth)
mrr_score = mean_reciprocal_rank(predictions, ground_truth)

# In kết quả
print(f"Top-1 Accuracy: {top1_acc:.4f}")
print(f"Top-5 Accuracy: {top5_acc:.4f}")
print(f"Top-10 Accuracy: {top10_acc:.4f}")
print(f"Top-20 Accuracy: {top20_acc:.4f}")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")

Top-1 Accuracy: 0.0000
Top-5 Accuracy: 0.0000
Top-10 Accuracy: 0.0000
Top-20 Accuracy: 0.0000
MAP: 0.0000
MRR: 0.0000


# Chọn danh sách file có liên quan đến câu summary trong quá khứ

In [102]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def bug_summary_similarity(df, input_bug_id):
    df_sorted = df.sort_values(by='report_time')

    input_bug_summary = df_sorted[df_sorted['bug_id'] == input_bug_id]['summary'].iloc[0]

    previous_bugs = df_sorted[df_sorted['report_time'] < df_sorted[df_sorted['bug_id'] == input_bug_id]['report_time'].iloc[0]]

    if previous_bugs.empty:
        return {}

    summaries = previous_bugs['summary'].tolist() + [input_bug_summary]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(summaries)

    similarity_scores = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1]).flatten()

    similarity_dict = dict(zip(previous_bugs['bug_id'], similarity_scores))

    return similarity_dict

In [103]:
def get_related_files(df, similarity_dict, threshold=0.5):
    # Filter bugs with similarity score higher than threshold
    relevant_bugs = {bug_id: score for bug_id, score in similarity_dict.items() if score >= threshold}

    if not relevant_bugs:
        return {}

    file_scores = {}

    for bug_id, score in relevant_bugs.items():
        files = df[df['bug_id'] == bug_id]['files'].iloc[0].split()
        for file in files:
            if file in file_scores:
                file_scores[file].append(score)
            else:
                file_scores[file] = [score]

    # Compute average relevance score for each file
    average_file_scores = {file: sum(scores) / len(scores) for file, scores in file_scores.items()}

    return average_file_scores

In [104]:
df = pd.read_excel(REPORT_PATH)

In [105]:
# related_files = get_related_files(df, bug_summary_similarity(df, 56012), 0.2)

In [106]:
# related_files

In [107]:
# print(type(56025))

In [108]:
for key, value in list_prediction.items():
    related_files = get_related_files(df, bug_summary_similarity(df, key), 0.05)
    print(value[0])
    print(related_files)
    break


IndexError: list index out of range

# Tổng hợp kết quả

In [ ]:
ALPHA = 0.8

In [ ]:
final_list_prediction = {}

# Duyệt qua list_prediction
for key, value in list_prediction.items():
    # Lấy related_files
    related_files_original = get_related_files(df, bug_summary_similarity(df, key), 0.2)
    related_files = {doc_id.replace('/', '\\'): sim for doc_id, sim in related_files_original.items()}
    # Chuyển value (list các tuple) thành dict để dễ xử lý
    value_dict = {doc_id: sim for doc_id, sim in value}

    # Kết hợp value_dict và related_files
    combined_dict = {}

    # Thêm tất cả từ value_dict
    for doc_id, sim in value_dict.items():
        combined_dict[doc_id] = sim

    # Cộng similarity nếu doc_id trùng với related_files
    for doc_id, sim in related_files.items():
        if doc_id in combined_dict:
            # Nếu trùng, cộng similarity (chuyển np.float64 về float để đồng nhất)
            combined_dict[doc_id] = ALPHA * combined_dict[doc_id] + (1-ALPHA) * float(sim)
        else:
            # Nếu không trùng, thêm mới
            combined_dict[doc_id] = float(sim)

    # Chuyển combined_dict thành list các dict và sắp xếp
    final_value = [
        {'doc_id': doc_id, 'similarity': sim}
        for doc_id, sim in sorted(combined_dict.items(), key=lambda x: x[1], reverse=True)
    ]

    # Lưu vào final_list_prediction
    final_list_prediction[key] = final_value

In [ ]:
len(list(final_list_prediction.values()))

In [ ]:
list_of_tuples = [
    [(item['doc_id'], item['similarity']) for item in inner_list]
    for inner_list in list(final_list_prediction.values())
]

In [ ]:
# Tính các chỉ số sau khi kết hợp Related_file
top1_acc = top_k_accuracy(list_of_tuples, ground_truth, k=1)
top5_acc = top_k_accuracy(list_of_tuples, ground_truth, k=5)
top10_acc = top_k_accuracy(list_of_tuples, ground_truth, k=10)
top20_acc = top_k_accuracy(list_of_tuples, ground_truth, k=20)
map_score = mean_average_precision(list_of_tuples, ground_truth)
mrr_score = mean_reciprocal_rank(list_of_tuples, ground_truth)

# In kết
print(f"Top-1 Accuracy: {top1_acc:.4f}")
print(f"Top-5 Accuracy: {top5_acc:.4f}")
print(f"Top-10 Accuracy: {top10_acc:.4f}")
print(f"Top-20 Accuracy: {top20_acc:.4f}")
print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")